Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Inventory Functions
---
There are two functions contained in this notebook. Adjusting the CoolerItemBalance table to update the product inventory based on what picklists are processed, 
and code to generate a projected restock date for each cooler based on the Many Models Machine Learning prediction of future demand based on data collected from 
the IoT system (see the demand_forecasting notebook and related documentation). As with all of these notebooks, this is desined to be run on an appropriate cadance 
for example using [Synapse pipelines](https://docs.microsoft.com/en-us/azure/synapse-analytics/get-started-pipelines) 

## Adjusting the CoolerItemBalance table

### 1.0 Imports and initalization

#### 1.1 Configuration

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import *
from notebookutils import mssparkutils
import json
sc = spark.sparkContext

synapse_account_name = '<Name of your Synapse Account>'
data_lake_account_name = '<Data Lake Account Name>' # Synapse Workspace ADLS
file_system_name = '<Data Lake Container Name>'
database_name = '<Database Name>'

spark.conf.set("spark.storage.synapse.linkedServiceName", f"{synapse_account_name}-WorkspaceDefaultStorage")
spark.conf.set("fs.azure.account.oauth.provider.type", "com.microsoft.azure.synapse.tokenlibrary.LinkedServiceBasedTokenProvider")


#### 1.1 Get data sets

In [ ]:
cooler_df = spark.read.parquet(f'abfss://{file_system_name}@{data_lake_account_name}.dfs.core.windows.net/{database_name}/cooler')
cooler_ids = [wh[0] for wh in cooler_df.select('CoolerId').collect()]
picklist_df = spark.read.parquet(f'abfss://{file_system_name}@{data_lake_account_name}.dfs.core.windows.net/{database_name}/picklist')
picklistitem_df = spark.read.parquet(f'abfss://{file_system_name}@{data_lake_account_name}.dfs.core.windows.net/{database_name}/picklistitem')
iotinventoryaction_df = spark.read.parquet(f'abfss://{file_system_name}@{data_lake_account_name}.dfs.core.windows.net/{database_name}/iotinventoryaction')
inventoryprojected_df = spark.read.parquet(f'abfss://{file_system_name}@{data_lake_account_name}.dfs.core.windows.net/{database_name}/inventoryprojected')
cooleritembalance_df = spark.read.parquet(f'abfss://{file_system_name}@{data_lake_account_name}.dfs.core.windows.net/{database_name}/cooleritembalance')
item_df = spark.read.parquet(f'abfss://{file_system_name}@{data_lake_account_name}.dfs.core.windows.net/{database_name}/item')
restockprojected_df = spark.read.parquet(f'abfss://{file_system_name}@{data_lake_account_name}.dfs.core.windows.net/{database_name}/restockprojected')

### 2.0 Adjust CoolerItemBalance table
- This tasks takes the latest items from the PickList data and updates the CoolerItemBalance table

#### 2.1 Get last update date for each sku

In [ ]:
w = Window.partitionBy('CoolerId','ItemSku').orderBy(desc('Timestamp'))
df = cooleritembalance_df.withColumn('Rank',dense_rank().over(w))

last_item_balance_df = df.filter(df.Rank == 1).drop(df.Rank).orderBy('CoolerId','ItemSku')
last_item_balance_df.show()

#### 2.2 Get overall last update from CoolerItemBalance

In [ ]:
#last_update = last_item_balance_df.select('Timestamp').orderBy(desc('Timestamp')).head()[0]

w = Window.orderBy(desc('Timestamp'))
df = last_item_balance_df.withColumn('Rank',dense_rank().over(w))

last_update = df.filter(df.Rank == 1).drop(df.Rank).select('Timestamp')[0]
print(last_update)

#### 2.3 Get the sum of items for the period between last item input into CoolerItemBalance and current

In [ ]:
items_removed_df = picklist_df.join(picklistitem_df, 'PickListId')

items_since_update_df = items_removed_df.filter(f"PickListFulfilledTimestamp > timestamp'{last_update}'")

items_current_sum = items_since_update_df.groupBy('CoolerId', 'ItemSku').agg(sum('ItemQuantity').alias('SumQuantity'),max('PickListFulfilledTimestamp').alias('PickListFulfilledTimestamp'))
items_current_sum.show()


#### 2.4 Write update back to CoolerItemBalance

In [ ]:
# write update back to CoolerItemBalance
wib_entry_df = items_current_sum.join(last_item_balance_df.alias("b"), 
    (items_current_sum.CoolerId == last_item_balance_df.CoolerId) & (items_current_sum.ItemSku == last_item_balance_df.ItemSku)).withColumn(
        'ActualItemQuantity',
        last_item_balance_df.ActualItemQuantity - items_current_sum.SumQuantity).select("b.CoolerId","b.ItemSku",col('PickListFulfilledTimestamp').alias('Timestamp'),"ActualItemQuantity")

wib_entry_df.write.insertInto(f"{database_name}.cooleritembalance")

wib_entry_df.show()


## Get Projected Restock Dates
-  Get latest ActualItemQuantity and TS from CoolerItemBalance
- Join present with projected from RestockProjected after this TS
- Loop decrementing per SKU until MinimumStockQuantity from Item
- Move current to previous in RestockProjected
- insert new projected in RestockProjected

### 1.0 Add cumulative column to inventoryprojected grouping by cooler and sku sorted by timestamp

In [ ]:
cumSum = inventoryprojected_df \
    .withColumn("cumulativeSum", sum(inventoryprojected_df["ItemQuantity"]) \
    .over( Window.partitionBy("CoolerId", "ItemSku").orderBy("PickListFulfilledTimestamp")))

cumSum.show()

### 2.0 Join cumulative column with CoolerItemBalance and Item

In [ ]:
# join cooleritembalance and item on sku
# select sku, ActualItemQuantity, and MinimumStockQuantity for each sku
# left join with cumSum on Item Sku
w = Window.partitionBy('CoolerId','ItemSku').orderBy(desc('Timestamp'))
df1 = cooleritembalance_df.join(item_df.alias('b'), 'ItemSku') \
    .withColumn('Rank',dense_rank() \
    .over(w))

item_info_df = df1.filter(df1.Rank == 1) \
    .drop(df1.Rank) \
    .orderBy('CoolerId','ItemSku')\
    .select('CoolerId','ItemSku', 'MinimumStockQuantity', 'ActualItemQuantity')

df2 = cumSum.join(item_info_df.alias('b'), ((cumSum.CoolerId == item_info_df.CoolerId) & (cumSum.ItemSku == item_info_df.ItemSku))) \
    .select('b.coolerId', 'b.ItemSku', 'PickListFulfilledTimestamp', 'cumulativeSum', 'ActualItemQuantity', 'MinimumStockQuantity')

df2.show()

### 3.0 Figure out dates for each sku by cooler

In [ ]:
w = Window.partitionBy('CoolerId').orderBy('PickListFulfilledTimestamp')
df3 = df2.filter((df2['ActualItemQuantity'] - df2['cumulativeSum']) < df2['MinimumStockQuantity']) \
    .withColumn('Rank',dense_rank() \
    .over(w))

df4 = df3.filter(df3.Rank == 1) \
    .drop(df3.Rank).select('CoolerId',col('PickListFulfilledTimestamp') \
    .alias('ProjectedDateTime'), ) \
    .dropDuplicates()

df4.show()


### 4.0 Insert into restockprojected

In [ ]:
df5 = df4.alias('a') \
    .join(restockprojected_df.alias('b'), 'CoolerId', 'left') \
    .select('CoolerId','a.ProjectedDateTime', col('b.ProjectedDateTime').alias('PreviousProjectedDateTime'))

df5 = df5.withColumn('ProjectedDateTime', to_timestamp('ProjectedDateTime')) \
    .dropDuplicates()
    
df5.write.insertInto(f"{database_name}.restockprojected")
    
df5.show()
    